In [1]:

from utils import *
from model import * 
json_file_path= "/Users/hwany/Documents/long_term_memory/train3537-3.json"
dialogs = process_dialogs(json_file_path, 2)
chunks = create_multiple_chunks(dialogs, 1, 1)
cache = create_vector_db(chunks)

/opt/homebrew/Caskroom/miniconda/base/envs/dacon/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
Creating Chunks: 100%|██████████| 25/25 [00:26<00:00,  1.07s/it]


In [17]:
for dialog in dialogs:
    print(dialog)

time: 2020/06/15/14:00
Speaker1: I need some advice on where to go on vacation, have you been anywhere lately?
Speaker2: I have been all over the world. I'm military.

time: 2020/06/15/14:00
Speaker1: That is good you have alot of travel experience
Speaker2: Sure do. And a lot of experience blowing things up! Haha. Bora bora is nice.

time: 2020/06/15/14:00
Speaker1: I've been working non stop crazy hours and need a break.
Speaker2: The best breaks are spent with cute cuddly kittens.

time: 2020/06/15/14:00
Speaker1: Bora bora sounds nice, you have been there before?
Speaker2: Nope... Just sounds nice, and repetitive. Bora... Bora. Ha!

time: 2020/06/15/14:00
Speaker1: Kittens really? I rather be at the beach.
Speaker2: Only if the beach was covered in kittens!

time: 2020/06/15/14:00
Speaker1: That would be a sight to see.
Speaker2: Or maybe brownies... I love chocolate.

time: 2020/06/15/14:00
Speaker1: I love brownies too but I haven't quite perfected mine yet.
Speaker2: Well I'm av

In [21]:
for chunk in chunks:
    print(chunk.page_content)
    print(chunk.metadata)

[2020/06/15/14:00] Speaker 1 seeks vacation advice, while Speaker 2 shares their extensive travel experience due to military service.
{'dialog': "time: 2020/06/15/14:00\nSpeaker1: I need some advice on where to go on vacation, have you been anywhere lately?\nSpeaker2: I have been all over the world. I'm military.\n", 'ids': 0, 'ref': 0, 'score': 0}
[2020/06/15/14:00] Speaker 1 acknowledges Speaker 2's travel experience, to which Speaker 2 humorously adds their experience with explosives and mentions Bora Bora as a nice destination.
{'dialog': 'time: 2020/06/15/14:00\nSpeaker1: That is good you have alot of travel experience\nSpeaker2: Sure do. And a lot of experience blowing things up! Haha. Bora bora is nice.\n', 'ids': 1, 'ref': 0, 'score': 1}
[2020/06/15/14:00] Speaker 1 expresses the need for a break due to working long hours, while Speaker 2 suggests that spending time with cute cuddly kittens is the best way to relax.
{'dialog': "time: 2020/06/15/14:00\nSpeaker1: I've been workin

In [6]:
query = {
    "time": "2020/6/21/20:00",
    "text": f"Speaker1: What did you do yesterday?"
}

is_needed_memory = int(check_memory(query["text"]))
new_query = f"[{query['time']}][{query['text']}]"
print("new_query: ",new_query)
print("is_memory_required: ", is_needed_memory)

# 만약 필요하다면 그냥 LLM 없이 time_stamp만 추가해서 쿼리 
# LLM의 king + man = queen 같은 성능을 바랬음
if is_needed_memory == 1:
    results = cache.similarity_search(new_query,5)
    for d in results:
        print(d.page_content)
else:
    print("n")

new_query:  [2020/6/21/20:00][Speaker1: What did you do yesterday?]
is_memory_required:  “[2020/6/20][]”
[2020/6/20 20:00] Speaker 2 discusses a friend's property in Florida that needs work but was purchased at a good price, and inquires about the safety of the new home.
[2020/06/17/14:00] Speaker 1 shares an experience about a cousin's house with aluminum wiring that required costly updates, and Speaker 2 acknowledges the importance of addressing home issues to prevent future problems.
[2020/06/15/14:00] Speaker 1 acknowledges Speaker 2's travel experience, and Speaker 2 humorously mentions their experience with explosives while recommending Bora Bora as a nice destination.
[2020/06/15/14:00] Speaker 1 expresses the need for a break due to working long hours, while Speaker 2 suggests that spending time with cute cuddly kittens is the best way to relax.
[2020/06/17/14:00] Speaker 1 inquires about Speaker 2's military status, and Speaker 2 confirms they have completed their service and 

In [24]:

query = {
    "time": "2020/6/21/20:00",
    "text": f"Speaker1: What did you do yesterday?"
}
query = f"[{query['time']}][{query['text']}]"
rewrited_query = query_rewriter(query)
print("query: ", query)
print("rewrited_query: ", rewrited_query)

if rewrited_query != "None":
    results = cache.similarity_search(rewrited_query,5)
    
    context = ""
    for idx, d in enumerate(results):
        context += f"{idx+1}: \n {d.page_content}\n"

print(context)

query:  [2020/6/21/20:00][Speaker1: What did you do yesterday?]
rewrited_query:  “[2020/6/20][]”
1: 
 [2020/06/17/14:00] Speaker 1 inquires about Speaker 2's military status, and Speaker 2 confirms they have completed their service and now work outside the military.
2: 
 [2020/6/20 20:00] Speaker 2 inquires if Speaker 1 still works as an electrician and offers to help with a wiring problem.
3: 
 [2020/06/20/14:00] Speaker 1 shares that they have booked a week-long vacation from work next month, and Speaker 2 expresses interest by asking where they are going.
4: 
 [2020/6/20] Speaker 2 discusses a friend's property in Florida that needs work but was purchased at a good price, and inquires about the safety of the new home.
5: 
 [2020/6/20/20:00] Speaker 2 expresses excitement about a planned trip and offers to cover the costs, then asks where Speaker 1's cousin lives.



In [16]:
validated_context = validater(context, rewrited_query).replace(",","")
idx_list = [int(x)-1 for x in validated_context.split() if x.isdigit()]
print(validated_context)
print(idx_list)

print("validated된 결과: ")
for idx in idx_list:
    print(results[idx].page_content)

Output: 2 3 4 5
[1, 2, 3, 4]
validated된 결과: 
[2020/6/20 20:00] Speaker 2 inquires if Speaker 1 still works as an electrician and offers to help with a wiring problem.
[2020/06/20/14:00] Speaker 1 shares that they have booked a week-long vacation from work next month, and Speaker 2 expresses interest by asking where they are going.
[2020/6/20] Speaker 2 discusses a friend's property in Florida that needs work but was purchased at a good price, and inquires about the safety of the new home.
[2020/6/20/20:00] Speaker 2 expresses excitement about a planned trip and offers to cover the costs, then asks where Speaker 1's cousin lives.


In [17]:
from openai import OpenAI
client = OpenAI(api_key="")

def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

test_case = [
    "[2020/06/15/14:00][Speaker1 expresses their love for brownies but mentions they haven't perfected their recipe yet. Speaker2 offers to help by volunteering to taste test the brownies.]",
    "[2020/06/15/14:00][Speaker1 expresses a preference for the beach over kittens. Speaker2 humorously suggests a beach covered in kittens. Speaker1 agrees that it would be an interesting sight. Speaker2 then mentions a love for chocolate, suggesting brownies as an alternative.]",
    "[2020/06/17/14:00][Speaker1 asks if Speaker2 is still in the military. Speaker2 responds that they are no longer serving, having completed their full term, and now work outside the military. Speaker1 expresses admiration and asks what Speaker2 does now. Speaker2 explains that they work as an electrical engineer for a private company, having been trained in the military for that role and discovering they enjoy and excel at it.]",
    "[2020/6/20][Speaker2 mentions taking two weeks off from work for the first time and asks Speaker1 how long they spent in the military. Speaker2 shares that they served in the military for 16 years and transitioned to a civilian job they love, expressing excitement about spending time on the beach in Bora Bora in a few years. Speaker2 then asks if Speaker1 still works as an electrician and offers to help fix a wiring problem for their cousin.]",
    "[2020/06/15/14:00][Speaker1 asks for vacation destination advice and inquires about recent travels. Speaker2 mentions their extensive travel experience due to being in the military and humorously adds that they have experience with explosives. Speaker2 recommends Bora Bora as a nice destination.]"
]


In [18]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(embedding1, embedding2):
    """
    두 벡터 간의 코사인 유사도를 계산하는 함수
    """
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

query_embedding = get_embedding("[2020/6/17][]")


for sample in test_case:
    sample_embedding = get_embedding(sample)
    print(compute_similarity(query_embedding, sample_embedding))


0.3065276073561899
0.2776960161830363
0.28218895787163384
0.26596997026217695
0.2572888855704737


In [21]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(embedding1, embedding2):
    """
    두 벡터 간의 코사인 유사도를 계산하는 함수
    """
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

query_embedding = get_embedding("[2020/6/18][Speaker1: What did you do yesterday?]")


for sample in test_case:
    sample_embedding = get_embedding(sample)
    print(compute_similarity(query_embedding, sample_embedding))



0.4108592778359869
0.3990480307571378
0.49456171259648884
0.4846625346958474
0.44613744145185386


In [22]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(embedding1, embedding2):
    """
    두 벡터 간의 코사인 유사도를 계산하는 함수
    """
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

query_embedding = get_embedding("[2020/6/21][Speaker1: What did you do four days ago?]")


for sample in test_case:
    sample_embedding = get_embedding(sample)
    print(compute_similarity(query_embedding, sample_embedding))

0.39743531926106823
0.3846414687548988
0.466194241331776
0.5138105545704537
0.45551629917546654
